In [ ]:
#!pip install --upgrade Flask pyngrok flask_ngrok2

# Public Service Publication
If you want to publish the portal for public demonstration, access the website (https://dashboard.ngrok.com/get-started/your-authtoken) register and generate an authentication token.

Otherwise, go directly to the next section

In [ ]:
#!ngrok authtoken YOUR_TOKEN
!ngrok authtoken YOUR_TOKEN

In [ ]:
from pyngrok import ngrok

port = 5000
public_url=ngrok.connect(port).public_url

print(public_url)

# Functions

In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np

img_height, img_width = 180,180

def predict(a,thres=0.5):
    if a>= thres:
        return 1
    else:
        return 0

def fresh_prediction(img_url):
    model = keras.models.load_model('model/fresh_prediction_ver3.keras')
    img = cv2.imread(img_url) 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
    img = cv2.resize(img, (img_height, img_width))
    prediction = model.predict(np.expand_dims(img, axis=0))
    probability = prediction[0][0]
    if predict(probability) == 1:
        is_fresh = 'Fresh'
    else:
        is_fresh = 'Rotten'
        probability = 1-probability
    return is_fresh,probability

def class_prediction(img_url):
    img_height,img_width = 180,180
    model = keras.models.load_model('model/class_prediction_ver2.keras')
    img = cv2.imread(img_url) 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
    img = cv2.resize(img, (img_height, img_width))
    class_prediction = model.predict(np.expand_dims(img, axis=0))
    class_probabilities = class_prediction[0]
    probability = max(class_probabilities)
    class_type = None
    if class_probabilities[0] == probability:
        class_type = 'Peach'
    elif class_probabilities[1] == probability:
        class_type='Pomegranate'
    else:
        class_type='Strawberry'
    return class_type,probability

# Local Service Publication

In [ ]:
import os
import base64
from datetime import datetime
from flask import *
from werkzeug.utils import secure_filename

UPLOAD_FOLDER = os.path.join('static', 'taken_image')

port = 5000

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

upload_result=[]

# Ensure the folder exists
if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])
    
@app.route('/')
def home_page():
    return render_template('home_page/index.html')

@app.route('/one_prediction_result')
def one_prediction_result():
    return render_template('one_prediction_result/index.html',upload_result=upload_result)

@app.route('/batch_prediction_result')
def batch_prediction_result():
    return render_template('batch_prediction_result/index.html',upload_result=upload_result)

@app.route('/camera_capture')
def camera_capture():
    return render_template('camera_capture/index.html')

@app.route('/take_picture', methods=['POST'])
def take_picture():
    if 'photoInput' in request.form:
        data_url = request.form['photoInput']
        encoded_data = data_url.split(',')[1]
        decoded_data = base64.b64decode(encoded_data)
        
        # Generate a unique file name using current timestamp
        timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
        file_name = f'photo_{timestamp}.jpg'

        folder_path = app.config['UPLOAD_FOLDER']
        os.makedirs(folder_path, exist_ok=True)
        file_path = os.path.join(folder_path, file_name)
        
        # Save the decoded image data to the specified file path
        with open(file_path, 'wb') as f:
            f.write(decoded_data)
        
        #Apply the model to image
        global upload_result
        upload_result = []
        class_type,class_probabilities=class_prediction(file_path)
        fresh_predict,fresh_probability=fresh_prediction(file_path)
        uploaded_image_url = request.url_root + file_path
        upload_result.append({
            'uploaded_image_url': uploaded_image_url,
            "class_probabilities": str(class_probabilities), 
            "class_type": class_type, 
            "fresh_probability": str(fresh_probability),
            "fresh_predict": fresh_predict
        })
        print(upload_result)
        # Redirect to a success page or return a success message
        return redirect('/one_prediction_result')
    return 'Invalid request.'
#upload 1 photo

ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return 'No file part'
    
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'
    timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
    file_name = f'photo_{timestamp}.jpg'
    
    # Specify the path where you want to save the photo
    folder_path = app.config['UPLOAD_FOLDER']
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, file_name)
    file.save(file_path)
    
    #Apply the model to image
    global upload_result
    upload_result = []
    class_type,class_probabilities=class_prediction(file_path)
    fresh_predict,fresh_probability=fresh_prediction(file_path)
    uploaded_image_url = request.url_root + file_path
    upload_result.append({
        'uploaded_image_url': uploaded_image_url,
        "class_probabilities": str(class_probabilities), 
        "class_type": class_type, 
        "fresh_probability": str(fresh_probability),
        "fresh_predict": fresh_predict
    })
    return jsonify(upload_result),200

@app.route('/upload_batch', methods=['POST'])
def upload_files():
    if 'files[]' not in request.files:
        return 'No file part', 400

    files = request.files.getlist('files[]')
    photo_num = 0
    file_names = []
    global upload_result
    upload_result = []
    for file in files:
        if file.filename == '':
            continue  # Skip empty filenames
        print(file)
        photo_num += 1
        timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
        file_name = f'photo_{photo_num}_{timestamp}.jpg'
        
        folder_path = app.config['UPLOAD_FOLDER']
        os.makedirs(folder_path, exist_ok=True)
        file_path = os.path.join(folder_path, file_name)
        file.save(file_path)
        file_names.append(file_name)
        
        #Apply the model to image
        class_type,class_probabilities=class_prediction(file_path)
        fresh_predict,fresh_probability=fresh_prediction(file_path)
        
        uploaded_image_url = request.url_root + file_path
        upload_result.append({
            'uploaded_image_url': uploaded_image_url,
            "class_probabilities": str(class_probabilities), 
            "class_type": class_type, 
            "fresh_probability": str(fresh_probability),
            "fresh_predict": fresh_predict
        })
    
    return file_names

if __name__ == '__main__':
    app.run(port = port)